<div style="border:solid Chocolate 2px; padding: 40px">

<b>Привет, Мария!👋</b>

Меня зовут Зуева Юлия, можно просто Юля 😏 Я буду ревьюером твоего проекта. Предлагаю обращаться на "ты", но если это неудобно, пожалуйста, скажи об этом!
    
В первую очередь хочу тебя поздравить с Новым годом и пожелать огромных успехов в обучении и карьерных достижений! 🎄🎅🏻 Ну а теперь вернемся к проекту, моя основная цель - помочь тебе стать компетентным Data Scientist'ом: вместе преодолеть трудности и еще больше развить твои сильные навыки 💪 В твоей работе я оставила комментарии, которые должны в этом помочь. Для твоего удобства комментарии раскрашены разными цветами: 
<br/>
<div class="alert alert-success">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3>
    
<b>Все отлично!👍:</b> Решение полностью правильное, или я посчитала нужным тебя за что-то похвалить :) На такие решения можно опираться в будущих проектах.
</div>

<div class="alert alert-warning">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3>
    
<b>Некоторые замечания и рекомендации💡:</b> Решение станет еще лучше, если внести небольшие коррективы. Ты можешь учесть эти комментарии при выполнении будущих заданий или доработать проект сейчас (но это не обязательно).
</div>
<div class="alert alert-block alert-danger">
<h3> Комментарий ревьюера <a class="tocSkip"></h3>
 
<b>На доработку🤔:</b> Решение требует существенной переработки. Напомню, что проект не может быть принят с первого раза, если ревью содержит комментарии, рекомендующие доработать шаги.
</div>
<br/>
Чтобы наша с тобой работа была еще эффективнее, ты точно так же можешь оставлять комментарии и задавать мне вопросы. Для таких случаев, пожалуйста, выбери собственное оформление комментариев, которое будет отличаться от моего, чтобы не создавалась путаница. По этой же причине попрошу тебя не удалять и не изменять мои комментарии. Все это позволит лучше ориентироваться в нашем с тобой диалоге, а повторную проверку (в случае ее необходимости) выполнять оперативнее 🚀

Не спеши расстраиваться, если что-то не удалось с первого раза. Вместе мы обязательно разберемся со всеми трудностями! 🤓 

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<div class="alert alert-warning">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3>
    
<b>Некоторые замечания и рекомендации💡:</b>  Очень хорошо, что оставляешь в начале проекта его описание! Любой, кто захочет взглянуть на работу, сможет сразу погрузиться в суть задачи. Будет немного лучше, если ты его чуть-чуть перефрзируешь, изложишь задачу своми словами, так работа станет похожа на настоящий исследовательский проект, а не выполненное домашнее задание :)

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.io as pio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [2]:
d0 = pd.read_csv('/datasets/geo_data_0.csv')

In [3]:
d0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
d0.duplicated().sum()

0

In [5]:
d1 = pd.read_csv('/datasets/geo_data_1.csv')

In [6]:
d1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
d1.duplicated().sum()

0

In [8]:
d2 = pd.read_csv('/datasets/geo_data_2.csv')

In [9]:
d2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
d2.duplicated().sum()

0

In [11]:
d0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [12]:
d0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [13]:
d1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [14]:
d2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Данные без пропускова и дубликатов.
Все признаки числовые, категориальных нет. Отделим признаки от целевого и разобъем выборки:

<div class="alert alert-success">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3> 
    
<b>Все отлично! 👍:</b> Супер, с данными познакомились, теперь смело можем двигаться дальше! 

<div class="alert alert-warning">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3>
    
<b>Некоторые замечания и рекомендации💡:</b> 

* По нескольким строкам (тем более ты  смотришь их только для одного региона) достаточно сложно понять, что происходит внутри всего датасета, вдруг наши данные загрузились криво, или там какие-то прям очевидные сильно выделяющиеся аномалии, которые будут мешать процессу обучения. Чтобы это проверить, важно изучать распределения переменных или хотя бы воспользоваться методом describe.    
    
    
* Поскольку дальше в работе мы планируем использовать линейную регрессию, было бы интересно предварительно провести корреляционный анализ и посмотреть, как переменные взаимосвязаны между собой. Это сможет дать некоторые инсайты о том, что же повлияло на качество предсказаний.

<div class="alert alert-success">
<b>Комментарий ревьюера 2:</b> 😉

In [15]:
target_0 = d0['product']
features_0 = d0.drop(['id', 'product'], axis=1)



features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)


target_1 = d1['product']
features_1 = d1.drop(['id', 'product'], axis=1)
 
    

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)


target_2 = d2['product']
features_2 = d2.drop(['id', 'product'], axis=1)


features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

In [16]:
d0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<div class="alert alert-block alert-danger">
<h3> Комментарий ревьюера<a class="tocSkip"></h3>
 
<b>На доработку🤔:</b> 
    
* На графики всегда необходимо добавлять заголовки, подписи осей и понятную легенду (если используются разные цвета или маркеры, сейчас лейблы легенды нам совсем ничего не говорят об изображении), чтобы график мог существовать в отрыве от кода (например, в презентации) или на случай не разбирающихся в коде заказчиков. 
    
    
* Обрати внимание, что в твоей функции scaler вне зависимости от переданных в функцию данных всегда обучается по одном и том же наборе features_0, то есть для двух остальных регионов скалирование происходит некорректно.
    
    
* Заметь, что ты скалируешь исходные данные, а далее в работе используешь неотскалированные выборки, которые были сформированы ранее.
    
    
* Отмечу, что стадартизировать данные до разделения на выборки не корректно, так как возникает утечка информации. Если мы сначала стандартизируем всю выборку и потом делим на тренировочную и валидационную, то получается, что тренировочная выборка отскалирована с учетом особенностей валидационной, поэтому модель заранее обучается этим особенностям и потом показывает хорошие результаты на валидации. То есть мы получаем неоправданное смещение метрики качества в лучшую сторону. Причем чем больше отличались друг от друга неотскалированные выборки, тем сильнее будет смещение. По смещенным оценкам нельзя принимать какие-то решения. 

v1 исправлено
убрала стандартизацию вообще, потому что после нее модель стала плыть и выдавать дикие значения

<div class="alert alert-success">
<b>Комментарий ревьюера 2:</b> Скорее всего стандартизация была проведена некорректно, или проблема была в чем-то еще. Ниже с помощью формул я показывала, что корректно проведенная стандартизация не может никак отразиться на результатах.

<div class="alert alert-warning">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3>
    
<b>Некоторые замечания и рекомендации💡:</b> 
    
* На основании чего сделан вывод, что распределения стало лучше, если мы не видели исходную ситуацию.
    
    
* На самом деле на **простой** линейной регрессии стандартизация не окажет никакого влияния на предсказания, поскольку все изменения переменной уйдут в новые коэффициенты: 

    $y = \alpha + \beta x$ - без стандартизации

    $y = \alpha_{st} + \beta_{st} \frac{x-mean}{std} = (\alpha_{st} - \frac{\beta_{st}mean}{std}) + (\frac{\beta_{st}}{std}) x$ - со стандартизацией

    При этом $\alpha = (\alpha_{st} - \frac{\beta_{st}mean}{std})$ и $\beta = (\frac{\beta_{st}}{std})$.

    По ссылкам можно узнать, когда все-таки стандартизация крайне важна: <a href = 'https://www.listendata.com/2017/04/how-to-standardize-variable-in-regression.html'>тык</a>, <a href = 'https://www.statlect.com/fundamentals-of-statistics/linear-regression-with-standardized-variables'>тык</a>.

    Одно из преимуществ стандартизации для линейной регрессии: мы можем по величине полученных коэффициентов судить о важности соотвествующих признаков (больше коэффициент - больше значимость). Без стандартизации такая интерпретация невозможна, так как каждый коэффициент в себя включает еще и элемент масштаба своего признака. Второй случай, когда стандартизация необходима - линейные регрессии с **регуляризацией** (ридж, лассо, эластик нет), поскольку данные модели регулируют те самые коэффициенты, на которые воздействует стандартизация. Ну и на всякий случай отмечу, что из-за особенностей формата float в некоторых знаках после запятой отличия все-таки будут, что может оказаться критичным, например, на соревнованиях.

## Обучение и проверка модели

### Скважины региона 1

In [17]:
def modeling(features, target, features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    print('Оценка модели: ', model.score(features_valid, target_valid))
    scores = cross_val_score(model, features, target)
    print('Средняя оценка кросс-валидации: ', scores.mean())
    prediction = model.predict(features_valid)
    pm = prediction.mean()
    rmse = mean_squared_error(target_valid, prediction) ** 0.5
    print('Предсказание модели в диапазоне rmse: ', pm, '+-', rmse)
    print('Реальное среднее: ', target_valid.mean())

In [18]:
modeling(features_0, target_0, features_train_0, target_train_0, features_valid_0, target_valid_0)

Оценка модели:  0.27994321524487786
Средняя оценка кросс-валидации:  0.27549130726904475
Предсказание модели в диапазоне rmse:  92.59256778438035 +- 37.5794217150813
Реальное среднее:  92.07859674082927


Кросс-валидация показала, что реальная точность модели чуть хуже

В среднем, предсказание модели оказалось очень точным

### Скважины региона  2

In [19]:
modeling(features_1, target_1, features_train_1, target_train_1, features_valid_1, target_valid_1)

Оценка модели:  0.9996233978805127
Средняя оценка кросс-валидации:  0.9996243728923553
Предсказание модели в диапазоне rmse:  68.728546895446 +- 0.893099286775617
Реальное среднее:  68.72313602435997


Удивительно высокая точность предсказания, даже не смотря на кросс-валидацию

Запасы этой скважины предсказаны точно, при этом их объем меньше, чем у предыдущей скважины

### Скважины региона  3

In [20]:
modeling(features_2, target_2, features_train_2, target_train_2, features_valid_2, target_valid_2)

Оценка модели:  0.20524758386040443
Средняя оценка кросс-валидации:  0.1987156246205129
Предсказание модели в диапазоне rmse:  94.96504596800489 +- 40.02970873393434
Реальное среднее:  94.88423280885438


Очень плохая предсказуемость модели

Объем и разброс предсказания запасов этой скважины почти такие же, как у первой

<div class="alert alert-success">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3> 
    
<b>Все отлично! 👍:</b> Обучение и валидацию проводишь верно! Круто, что создаешь специальную функцию для выполнения повторяющихся действий, это позволяет сделать код более аккуратным и эффективным. С этого момента бывают подводные камни. Мы получили валидационные подвыборки с помощью `train_test_split`, индексы при этом будут хаотичны в них (соответсвовать исходному датасету), а предсказания у нас упорядочены. За этим надо следить в данном проекте, чтобы потом правильно ставилось соответствие таргета и предсказания, именно для одной и той же скважины.

<div class="alert alert-warning">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3>
    
<b>Некоторые замечания и рекомендации💡:</b> 
    
* Кросс-валидация - это интсрумент для получения более независимых метрик (усредненных между выборками), чтобы **выбор между моделями** был более надежным. Поскольку в данном проеткте у нас не стоит задачи подбирать модель, в кросс-валидации необходимости нет.
    
    
* На будущее отмечу, что если бы ты для моделирования использовала отскалированную тренировочную выборку, то ее было бы некорректно использовать для кросс-валидации по той же причине утечки: если мы сначала стандартизируем всю тренировочную выборку и потом делим на подтренировочную и валидационную, то получается, что подтренировочная выборка отскалирована с учетом особенностей валидационной, поэтому модель заранее обучается этим особенностям и потом показывает хорошие результаты на валидации. 
    
    Чтобы этого не допускать, скалировани тренировочной выборки необходимо проводить не до кросс-валидации, а во время (то есть сначала будут формироваться подвыборки, а затем производиться скалирование). Технически это можно реализовать с помощью пайплайнов (функция для создания порядка выполнения методов), например так:

    ```python
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(StandardScaler(), ml_model)
cross_val_score(pipeline, X_train, y_train)
    ```

## Подготовка к расчёту прибыли

In [21]:
min_barr_count = 10000000 / ( 450 * 200 )

<div class="alert alert-block alert-danger">
<h3> Комментарий ревьюера <a class="tocSkip"></h3>
 
<b>На доработку🤔:</b> Если в работе используются какие-то константные значения, то их лучше сохранять в отдельных переменных и потом к ним обращаться. Это позволяет избежать ошибок при редактировании проекта, то есть достаточно будет поменять значение только одной константы, чтобы поменялись все завиясящие от нее расчеты. К тому же код будет читать гораздо удобнее, так как мы будем понимать, что означает каждая из констант (по голому числу в коде сложно быстро догадаться, что оно значит).

<div class="alert alert-danger">
<b>Комментарий ревьюера 2:</b> Это замечание так и не было исправлено. Когда проекты станут в разы больше (в реальной практике) без созданных констант будет очень просото запутаться.

Почему не испралвено, я же создала константу min_barr_count 😢 

<div class="alert alert-danger">
<b>Комментарий ревьюера 3:</b> Это уже считается расчетной переменной, которую ты выводишь в проекте (это своего рода часть исследования) :) Под константами подразумеваются отдельные числа, которые ты сейчас использовала в своих расчетах, то есть, например, 10000000 - это константа бюджета и т.д.
    
Согласно стандартам оформления <a href = 'https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#section-29'>PEP-8</a> названия констант принято задавать заглавными буквами (например, BUDGET). Также на будущее скажу, что стандарты разработки рекомендуют объявлять их в начале пректа: сначала идут импорты, затем константы, потом пользовательские функции и классы (с комментариями, для чего каждый из созданных объектов предназначен). К исследовательским проектам это применяется реже, но в некоторых случаях действительно оказывается полезным. 
    
Далее везде в твоих исследовательских расчетах, функциях ты вместо числа используешь эту созданную константу BUDGET. Это удобно, так как и поменять все расчеты можно быстро, если вдруг придет заказчик и скажет, что у него есть теперь 15 млрд на разработку, а также и при чтении кода сразу понятно, что в расчетах используется бюджет, а не неведомое нам число.
    
Кстати, большие числа можно записывать либо через возведение в степень: 10**10 = 1e10, либо так: 10_000_000_000. Все это то же самое, что и 10000000000, но легче для восприятия.

Минимальное значение для окупаемости около 111 тыс. баррелей, что выше среднего для всех регионов

<div class="alert alert-success">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3> 
    
<b>Все отлично! 👍:</b> Все верно! Во всех регионах средняя концентрация продукта слишком низкая, чтобы выбирать точки случайно, поэтому нужен хороший механизм для отбора.

In [22]:
def profit(target, probabilities, count):
    target = target.reset_index(drop = True)
    probabilities = probabilities.reset_index(drop = True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450 * selected.sum() - 10000000

<div class="alert alert-warning">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3>
    
<b>Некоторые замечания и рекомендации💡:</b> Согласно рекомендациям по оформлению <a href = 'https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#section-18'>PEP-8</a> названия всех функций и переменных должны быть интуитивно понятны всем и отражать основной смысл. Revenue переводится как выручка, то есть сколько мы получили с продажи, в то время как profit переводится как прибыль, сколько заработали (выручка минус затраты, именно это мы и считаем).

<div class="alert alert-success">
<b>Комментарий ревьюера 2:</b> 😉

In [23]:
def calc(target, features_valid, features_train, target_train):
    model = LinearRegression()
    model.fit(features_train, target_train)
    target = target.reset_index(drop=True)
    probabilities = pd.Series(model.predict(features_valid))
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample, 200))

    fig = go.Figure()
    fig.add_trace(go.Box(y=values, name='Распределение прибыли'))
    fig.show("svg")        
    values = pd.Series(values)
    loss_proba = values[values < 0].count() / values.count()
    lower = values.quantile(q=0.025)
    upper = values.quantile(q=0.975)

    mean = values.mean()
    print("Средняя выручка:", '{:.1f}'.format(mean/1000), 'млрд')
    print("5%-квантиль:", '{:.1f}'.format(lower/1000), 'млрд')
    print("95%-квантиль:", '{:.1f}'.format(upper/1000), 'млрд')
    print("Вероятность убытков:", '{:.1f}'.format(loss_proba * 100), '%')


<div class="alert alert-success">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3> 
    
<b>Все отлично! 👍:</b> Ты верно делаешь, что при семплировании данных устанавливаешь параметр **replace = True**. Особенностью техники бутстреп является семплирование с возвратом, то есть наблюдение в выборку может попасть несколько раз. Это необходимо для того, чтобы добиться независимости при каждом выборе, иначе вероятность достать элемент из выборки будет с каждым разом увеличиваться. Причем чем больше размер подвыборки, тем выраженнее будет зависимость. Для подвыборки размера 1, например, в крайнем случае, мы возрат можем не делать, так уж и быть, а вот для подвыборки размером с семплируемую, без возврата ну никак не получится получить разнообразные варианты, а подвыборку большего размера вообще — не получить.
    
Все это может показаться немного странным для понимания: как будто одна скважина может буриться несколько раз. Но давай вспомним, что выборка - это лишь некоторое представление о генеральной совокупности. Вполне возможно, что где-то в регионе существуют и другие потенциальные точки добычи, о которых мы даже не подозреваем, но они являются частью генеральной сововкупности. В таком случае повторяющиеся наблюдения можно рассматривать как разные точки добычи из всей генеральной совокупности, просто они имеют близкие характеристики. 

<div class="alert alert-block alert-danger">
<h3> Комментарий ревьюера<a class="tocSkip"></h3>
 
<b>На доработку🤔:</b> 
    
* Поскольку мы вынуждены формировать выборку с возвращением (**replace = True**), в сабсемпле у нас образуются повторения, причем дублироваться будут и индексы. Затем мы формируем на основе него второй сабсемпл, и в нем тоже будут дублироваться индексы. Но давай посмторим, что происходит, когда мы пытаемся достать какие-то значения из вектора с дублирующимися индексами, побробуй выполнить следующий код:
    
    ```python
    t_subsample = pd.Series([5, 5, 7, 7], index = [0, 0, 1, 1])
    p_subsample = pd.Series([5.5, 5.5, 7.5, 7.5], index = [0, 0, 1, 1])

    # Пробуем достать одно значение
    print(t_subsample[0])

    # Пробуем достать все значения по исходным индексам, ожидаем увидеть 4 значения
    print()
    print(t_subsample[p_subsample.index])
    ```  
    Видим, что из-за дублирования количество отобранных значений разрастается. Это непременно будет приводить к завышению прибыли и занижению рисков. Чтобы избежать данной проблемы, нам нужно либо перед функцией прибыли сбросить у обоих сабсемплов индексы (чтобы каждое повторяющееся значения стало считаться уникальным за счет индекса, и при этом между векторами не нарушилась сопоставимость):

    ```python
    # Cбрасываем индексы
    t_subsample = t_subsample.reset_index(drop = True)
    p_subsample = p_subsample.reset_index(drop = True)

    # Пробуем достать значения снова
    print(t_subsample[0])
    print()
    print(t_subsample[p_subsample.index])
    ```  
    либо не сбрасывать индексы, но в функцию прибыли передавать полный валидационный вектор таргета, в котором индексы не дублируются и соотвествуют индексам предсказаний (обозначают одни и те же скважины), либо вовсе придумать способ как можно обойтись без индексов (такие есть, но быстро их объяснить не получится).
    
    
* Сейчас у тебя рассчитывается только среднее значение прибыли, левая граница доверительного интервала (нужна еще правая), и совсем не рассчитываются риски (заказчик запрашивал значение вероятности, с которой он может столкнуться с убытками).

v1 исправлено. сброс индексов добавила в профит

<div class="alert alert-success">
<b>Комментарий ревьюера 2:</b> Отлично! Теперь результаты не искажаются дублированием :)

<div class="alert alert-danger">
<b>Комментарий ревьюера 2:</b> Обрати внимание, что ты нашла 90% интервал вместо требуемого 95%.

v2 исправлено

<div class="alert alert-success">
<b>Комментарий ревьюера 3:</b> Супер!

<div class="alert alert-warning">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3>
    
<b>Некоторые замечания и рекомендации💡:</b> Чтобы сейчас не тратить время на получение предсказаний, их можно было бы сохранить во втором разделе с моделированием.

## Расчёт прибыли и рисков 

In [24]:
calc(target_valid_0, features_valid_0, features_train_0, target_train_0)

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
calc(target_valid_1, features_valid_1, features_train_1, target_train_1)

<div class="alert alert-block alert-danger">
<h3> Комментарий ревьюера <a class="tocSkip"></h3>

<b>На доработку🤔:</b> Перед отправкой проекта важно сбрасывать ядро (очищать память от созданных объектов) и заново запускать проект с самого начала, чтобы проверить на наличие ошибок в коде. Сейчас код не смог выполниться до конца.

<div class="alert alert-success">
<b>Комментарий ревьюера 2:</b> 😉

In [ ]:
calc(target_valid_2, features_valid_2, features_train_2, target_train_2)

Единственный подходящий регион под номером 1

<div class="alert alert-warning">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3>
    
<b>Некоторые замечания и рекомендации💡:</b> 

* Числа с длинной дробной частью воспринимаются не очень хорошо. Ее лучше округлять до нескольких знаков, можно с помощью того же <a href = 'https://queirozf.com/entries/python-number-formatting-examples'>форматирования</a>.
    
    
* Еще кстати очень удобно можно визуализировать результаты: строить распределения прибыли и добавлять на них вертикальные линии - границы доверительного интервала и границу безубыточности.

<div class="alert alert-block alert-danger">
<h3> Комментарий ревьюера <a class="tocSkip"></h3>

<b>На доработку🤔:</b> В конце выполнения работы важно оставлять итоговый резюмирующий вывод с обзором **всей** работы целиком. Иногда заказчики не просматривают весь проект, а обращаются прямо к выводам, поэтому в них важно отражать выжимку основных моментов, находок, и принятых тобою решений **по каждому** из разделов проекта.

## Вывод

Цель работы -- определение региона с наибольшей прибылью и наименьшими рисками. \
\
В исходных данных имелось три региона под номерами 0, 1 и 2. Для каждого была создана модель линейной регрессии, предсказывающая количество баррелей в скажине по ее параметрам. \
Самым предсказуемым оказался регион под номером 1. Высокая точность модели позволяет лучше всего оценить реальные запасы в скважинах. При этом в данном регионе средние объемы залежей меньше, чем в остальных.\
\
Было проиведено моделирование выбора лучших 200 скважин из 500 отобраных случайным образом. Результаты исследования показали: хоть регионы 0 и 2 потенциально могут принести большую прибыль (95% квантиль), они слишком непредсказуемы, и вероятность убытков в них велика. Только 1 регион имеет достаточно низкую вероятность убытков -- всего 1.5%. Именно он рекомендуется к выбору.


<div class="alert alert-success">
<b>Комментарий ревьюера 2:</b> Супер!

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован

<div style="border:solid Chocolate 2px; padding: 40px">

**Общий вывод по проекту**

Мария, спасибо за твой проект! У тебя замечательная работа! В тебе виден будущий профессионал своего дела :) 
    
Кратко пройдусь по впечатлениям. 

**Отмечу отдельные положительные моменты проекта🙂**:
 
* Прекрасное владение инструментами обработки данных!
* Умелое проведение машинного обучения!
* Потрясающие навыки использования функций для повышения эффективности работы!
* Дополнение своими идеями - очень круто!
* Все этапы работы проделаны очень осмысленно! Каждый шаг снабжен пояснениями и выводами!
* Работа имеет четкую структуру и выполнена аккуратно, проверять - одно удовольствие!

    
**На что стоит обратить внимание🤓**:
   
* Перед отправкой важно сбрасывать ядро и проверять код на наличией ошибок.
* На графики важно не забывать добавлять понятную сопроводительную информацию.
* При скалировании данных (и совмещении данной процедуры с кросс-валидацией) важно не допускать утечки информации.
* При использовании констант важно заводить для них отдельные переменные.
* При подсчете прибыли помним про индексы: важно, чтобы они не дублировались.
* Заказчик запрашивал более точное значение вероятности того, что прибыль меньше 0, а также 95% доверительный интревал.
* В конце работы важно оставлять резюмирующий вывод.
* Оставила для тебя также некоторые рекомендаций по улучшению оформления и эффективности кода.
    

**Желаю удачи и жду твой проект на повторное ревью!**😉
    
<img src="https://99px.ru/sstorage/86/2015/12/image_86271215043043632690.gif" />

<div class="alert alert-success">
<b>Комментарий ревьюера 3:</b> Приму проект, так как константы уже больше касаются оформления и не критично важны для результата, но в следующий раз все-таки не ленись их создавать, чтобы привыкнуть к этому и впоследствии не путаться в крупных проектах. Было очень приятно с тобой поработать :) Еще раз поздравляю с праздниками и желаю покорить все вершины! 💪